In [1]:
import intprim
from intprim.probabilistic_movement_primitives import *
import matplotlib.pyplot as plt
from intprim.util.kinematics import BaseKinematicsClass
from cem import CEM
from utils import *
import time
from pytorch_fid.fid_score import calculate_frechet_distance
from ppo_via import Actor
import argparse
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import os
from torch.distributions.multivariate_normal import MultivariateNormal
from matplotlib.patches import Rectangle

# state 是障碍物位置， 轨迹的起点终点固定

def get_args():
    parser = argparse.ArgumentParser(description="hyper parameters")
    parser.add_argument('--algo_name', default='PPO', type=str, help="name of algorithm")
    parser.add_argument('--train_eps', default=500, type=int, help="episodes of training")
    parser.add_argument('--test_eps', default=20, type=int, help="episodes of testing")
    parser.add_argument('--gamma', default=0.99, type=float, help="discounted factor")
    parser.add_argument('--mini_batch_size', default=5, type=int, help='mini batch size')
    parser.add_argument('--n_epochs', default=4, type=int, help='update number')
    parser.add_argument('--actor_lr', default=0.0003, type=float, help="learning rate of actor net")
    parser.add_argument('--critic_lr', default=0.0003, type=float, help="learning rate of critic net")
    parser.add_argument('--gae_lambda', default=0.95, type=float, help='GAE lambda')
    parser.add_argument('--policy_clip', default=0.2, type=float, help='policy clip')
    parser.add_argument('-batch_size', default=20, type=int, help='batch size')
    parser.add_argument('--hidden_dim', default=8, type=int, help='hidden dim')
    parser.add_argument('--device', default='cpu', type=str, help="cpu or cuda")
#     args = parser.parse_args()
    args = parser.parse_args(args=[])
    return args

cfg = get_args()

n_states=2
n_actions=3

actor =Actor(n_states, n_actions, cfg)
actor.load_checkpoint()

dataset= np.load('/home/zhiyuan/notebook_script/0930/l_shape.npy')
num_joints =2
basis_model = intprim.basis.GaussianModel(8, 0.1, ["x","y"])
promp = ProMP(basis_model)
Q = dataset.transpose(0,2,1)

for i in range(len(Q)):
	promp.add_demonstration(Q[i])
n_samples = 30
domain = np.linspace(0,1,100)

mean_margs =  np.zeros([2,100])
sigma_s = np.zeros([2,2,100])
# stdqs =  np.zeros([2,100])
for i in range(len(domain)):
    mu_marg_q, Sigma_marg_q = promp.get_marginal(domain[i])
    sigma_s[:,:,i] = Sigma_marg_q
    mean_margs[:,i] = mu_marg_q

# prepared for fid score
mu_1= mean_margs.T.reshape(-1)
sigma_1 = np.zeros([200,200])
for i in range(100):
    sigma_1[i*2:(i+1)*2,i*2:(i+1)*2] = sigma_s[:,:,i]


# state: start, end , obstcle   长和宽为一的正方形 只要确定左下角的x和y

# def get_reward(state, action):
#     via_point= np.zeros(3)
#     via_point[0]=action[0]
#     via_point[1]=action[1]
#     via_point[2]=action[2]
#     # via_point= action
#     old_promp=promp
#     via_point = via_point.reshape([-1,3])
#     t_cond = np.zeros(2+via_point.shape[0])
#     t_cond[0]=0
#     t_cond[-1] = 1 
#     for i in range(via_point.shape[0]):
#         t_cond[i+1]=via_point[i,2]

#     q_cond =np.zeros([2+via_point.shape[0],2])
#     q_cond[0]=mean_margs[:,0]
#     q_cond[-1]=mean_margs[:,-1]
#     # q_cond[0]= np.array([state[0],state[1]])
#     # q_cond[-1]=np.array([state[2],state[3]])
#     for i in range(via_point.shape[0]):
#         q_cond[i+1]=via_point[i,:2]
    
#     mu_w_cond_rec, Sigma_w_cond_rec=old_promp.get_basis_weight_parameters()

#     for i in range(t_cond.shape[0]):
#         mu_w_cond_rec, Sigma_w_cond_rec = old_promp.get_conditioned_weights(t_cond[i], q_cond[i], mean_w=mu_w_cond_rec, var_w=Sigma_w_cond_rec)

#     cond_traj = np.zeros([2,100])
#     sigma_con_array =np.zeros([2,2,100]) 
#     for i in range(len(domain)):
#         mu_marg_q_con, Sigma_marg_q_con = promp.get_marginal(domain[i], mu_w_cond_rec, Sigma_w_cond_rec)
#         sigma_con_array[:,:,i] = Sigma_marg_q_con
#         cond_traj[:,i] = mu_marg_q_con

#     limit = np.array([[state[0],state[0]+1],[state[1],state[1]+1]])   
#     # limit = np.array([[state[4],state[4]+1],[state[5],state[5]+1]])   
#     obs_dis= traj_rect_dist(cond_traj.T,  limit)  #new distance
#     orig_obs= traj_rect_dist(mean_margs.T,  limit) #old distance
#     obs_reward=obs_dis- orig_obs #new distance -old distance
#     # mu_1= mean_margs.T.reshape(-1)
#     mu_2= cond_traj.T.reshape(-1)
#     # sigma_1 = np.zeros([200,200])
#     sigma_2 = np.zeros([200,200])
#     for i in range(100):
#         # sigma_1[i*2:(i+1)*2,i*2:(i+1)*2] = sigma_s[:,:,i]
#         sigma_2[i*2:(i+1)*2,i*2:(i+1)*2] = sigma_con_array[:,:,i]
#     fid_cost= calculate_frechet_distance(mu_1,sigma_1,mu_2,sigma_2) *0.01

#     alif =0.1

#     reward =    obs_dis*(1-alif) -alif* fid_cost

#     return reward

In [2]:
# from matplotlib.patches import Rectangle
# fig, ax = plt.subplots()
# ax.axis([0, 10, 0, 10])
# plt.gca().set_aspect(1)
# ax.plot(mean_margs[0,:], mean_margs[1,:], 'g', alpha=0.3)

# #add rectangle to plor
# ax.add_patch(Rectangle((5, 3), 1, 1,
# # ax.add_patch(Rectangle((4, 5), 3, 3,
#              angle=0,
#              edgecolor = 'none',
#              facecolor = 'blue',
#              fill=True,
#              lw=1))


# plt.show()
# print(dataset.shape)

# def collision_detect(rect,traj):
#     if any(rect[0,0]<=p[0]<=rect[0,1] and rect[1,0]<=p[1]<=rect[1,1] for p in traj):
#         col =1 
#     else:
#         col=0
        
#     return col
# limit=np.array([[8,9],[8,9]])
# collision_detect(limit,mean_margs.T)

In [3]:
def get_via(limit):
    dist=actor(limit)
    via= dist.sample()
    
    return via.numpy() 
def plot_traj(limit):
    old_promp=promp

    via_point= get_via(limit)

    via_point = via_point.reshape([-1,3])
    t_cond = np.zeros(2+via_point.shape[0])
    t_cond[0]=0
    t_cond[-1] = 1
    for i in range(via_point.shape[0]):
        t_cond[i+1]=via_point[i,2]

    q_cond =np.zeros([2+via_point.shape[0],2])
    q_cond[0]= mean_margs[:,0]
    q_cond[-1]= mean_margs[:,-1]
    for i in range(via_point.shape[0]):
        q_cond[i+1]=via_point[i,:2]
    print(via_point)
    mu_w_cond_rec, Sigma_w_cond_rec=old_promp.get_basis_weight_parameters()

    for i in range(t_cond.shape[0]):
        mu_w_cond_rec, Sigma_w_cond_rec = old_promp.get_conditioned_weights(t_cond[i], q_cond[i], mean_w=mu_w_cond_rec, var_w=Sigma_w_cond_rec)
    cond_traj = np.zeros([2,100])
    sigma_con_array =np.zeros([2,2,100]) 
    upper_bound_con =  np.zeros([2,100])
    lower_bound_con =  np.zeros([2,100])

    for i in range(len(domain)):
        mu_marg_q_con, Sigma_marg_q_con = promp.get_marginal(domain[i], mu_w_cond_rec, Sigma_w_cond_rec)
        sigma_con_array[:,:,i] = Sigma_marg_q_con
        cond_traj[:,i] = mu_marg_q_con
        std_q_con = np.diagonal(Sigma_marg_q_con)** 0.5
        upper_bound_con[:,i] = mu_marg_q_con + std_q_con
        lower_bound_con[:,i] = mu_marg_q_con - std_q_con

  
    
    # %matplotlib
    fig, ax = plt.subplots()
    ax.plot(cond_traj[0,:], cond_traj[1,:], 'r', alpha=0.3)
    ax.plot(mean_margs[0,:], mean_margs[1,:], 'g', alpha=0.3)
    for i in range(t_cond.shape[0]):
        ax.plot(q_cond[i,0], q_cond[i,1],  marker="o", markersize=5)

    #add rectangle to plor
    ax.add_patch(Rectangle((limit[0],limit[1]), 1, 1,

                 angle=0,
                 edgecolor = 'none',
                 facecolor = 'blue',
                 fill=True,
                 lw=1))


    plt.show()

In [4]:
# # limit是tensor
start=mean_margs[:,0]
end=mean_margs[:,-1]
def env_reset():
    states = np.zeros(2)
    states[0]=np.random.uniform(2,6.5)
    states[1]=np.random.uniform(2,7.5)
    limit = np.array([[states[0],states[0]+1],[states[1],states[1]+1]])  
    d1= rect_dist(start,limit)
    d2= rect_dist(end,limit)
    while d1*d2 ==0:
        states[0]=np.random.uniform(2,6.5)
        states[1]=np.random.uniform(2,7.5)
        limit = np.array([[states[0],states[0]+1],[states[1],states[1]+1]])  
        d1= rect_dist(start,limit)
        d2= rect_dist(end,limit)
    return states



def sample_obst(num_ob):
    states=np.zeros([num_ob,2])
    for i in range(num_ob):
        states[i]=env_reset()       
    states = torch.from_numpy(states)
    return states

limits= sample_obst(10)
# print(limits)
for i in range(10):
    _limit = limits[i]
#     print(get_via(_limit))
    plot_traj(_limit)
    

ValueError: cannot reshape array of size 2 into shape (3)

In [1]:
fig, ax = plt.subplots()
ax.axis([1, 8, 1, 8])

ax.plot(5.5, 3.5,  marker="o", markersize=5)

ax.add_patch(Rectangle((5, 3), 1, 1,

             angle=0,
             edgecolor = 'none',
             facecolor = 'pink',
             fill=True,
             lw=1))


plt.show()

NameError: name 'plt' is not defined

In [9]:
def get_via(limit):
    dist=actor(limit)
    via= dist.sample()
    
    return via.numpy() 

In [5]:
limits= sample_obst(10)
limits

tensor([[4.7195, 2.7084],
        [4.7853, 2.7608],
        [2.5425, 3.2698],
        [5.8541, 3.2938],
        [5.1865, 7.4963],
        [3.0267, 5.8278],
        [3.8680, 6.1470],
        [2.9439, 6.6294],
        [3.0773, 3.3523],
        [4.7058, 5.4042]], dtype=torch.float64)

In [6]:
get_via(limits[0])

array([1.0538384 , 6.6123466 , 0.40394968], dtype=float32)

In [7]:
get_via(limits[4].float())

array([1.1272964 , 6.9630103 , 0.52531624], dtype=float32)

In [23]:
def env_reset():
    states = np.zeros(6)
    mu= np.array([2.57696126, 2.34334736, 6.38525582, 3.0674252])
    for i in range(4):
        states[i] = np.random.normal(loc=mu[i], scale=0.5)
    states[4]=np.random.uniform(0,7)
    states[5]=np.random.uniform(0,7)
    states[:4]=  np.clip(states[:4],  0, 8)
    start = states[0:2]
    end = states[2:4]
    limit = np.array([[states[4],states[4]+1],[states[5],states[5]+1]])  
    d1= rect_dist(start,limit)
    d2= rect_dist(end,limit)
    while d1*d2 ==0:
        for i in range(4):
            states[i] = np.random.normal(loc=mu[i], scale=0.5)
        states[4]=np.random.uniform(0,7)
        states[5]=np.random.uniform(0,7)
        states[:4]=  np.clip(states[:4],  0, 8)

        start = states[0:2]
        end = states[2:4]
        limit = np.array([[states[4],states[4]+1],[states[5],states[5]+1]])  
        d1= rect_dist(start,limit)
        d2= rect_dist(end,limit)

    return states
env_reset()

array([2.02505793, 2.11475211, 6.21437158, 2.86013626, 6.71227937,
       6.17218136])